## Code for labeling regions 

In [1]:
import json
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import re
from itertools import combinations

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [5]:
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

In [6]:
%run functions-py.py

## Making json file of guides

#### Guides for shen atlas
We convert the text files extracted from <a href="https://github.com/bioimagesuiteweb/bisweb/blob/devel/web/images/atlases/humanmni.json"> Bioimage Suite Web </a> into json files (when we copied them were not as pretty as now)

In [5]:
gui=["../Data/Parcellation/gui_lobes.txt","../Data/Parcellation/gui_brodlabels.txt","../Data/Parcellation/gui_networks.txt"]

In [ ]:
#guides is the dictionary of attributes 
guides=dict()
for file in gui:
    x=open(file,'r')
    lines=x.readlines()
    x.close()
    name=lines[0].split("=")[0].split()[0]
    guides[name]=dict()
    for i in range(1,(len(lines)-1)):
        v=lines[i].split()
        key=int(v[0].split(":")[0]) #the number
        ele=v[1].split("\"")[1]     #the name
        guides[name][key]=ele

In [ ]:
print(guides.keys())
guides["gui_Networks"]

In [ ]:
#now guides i in json file
with open("../Data/Parcellation/guides.json", "w") as write_file:
    json.dump(guides, write_file)

#### Guides for aal atlas

In [ ]:
#aal atlas from mricron
file="../../../Documentos/Software/mricron/Resources/templates/aal.nii.txt"
guide=dict()
x=open(file,'r')
lines=x.readlines()
x.close()
for i in range(0,(len(lines)-1)):
    v=lines[i].split(" ")
    key=int(v[0])
    ele=v[1]
    guide[key]=ele

In [ ]:
guide[0]='None'

In [ ]:
with open("../Data/Parcellation/aal_atlas.json", "w") as write_file:
    json.dump(guide, write_file)

## SHEN INFORMATION FROM BIOIMAGE

Checking information from json files.
The bioimage.json file is extracted from <a href="https://github.com/bioimagesuiteweb/bisweb/blob/devel/web/images/shen.json"> Bioimage web <a/>

In [9]:
file_rois="../Data/Parcellation/bioimage.json"
file_info="../Data/Parcellation/guides.json"

In [10]:
#Now the ROIS file, let's see
with open(file_rois, "r") as read_file:
    rois = json.load(read_file)

In [11]:
rois["rois"][257]

{'x': -12.52, 'y': 11.62, 'z': 8.68, 'attr': [19, 42, 10, 48, 9]}

In [12]:
#Run region_information 
region_information=set_dictionaries_rois(file_info,file_rois)

In [13]:
region_information(256)

{'MNI': [-10.85, 23.86, 9.64],
 'Lobe': 'L-Subcortical',
 'Network': 'Salience',
 'Network2': 'Basal Ganglia',
 'Brodmann Area': 'Unknown'}

## MNI to Atlas region

We associate the centroid (MNI coordinate) of each region with the name in region atlas (aal atlas)

In [14]:
%run functions-py.py

In [15]:
from nilearn import datasets,image
import nibabel as nib

The file of the Shen Atlas is extracted from <a href="https://www.nitrc.org/frs/?group_id=51"> NITRC site <a/>

In [3]:
path_atlas="../Data/Parcellation/shen_2mm_268_parcellation.nii.gz"
atlas=nib.load(path_atlas)

In [4]:
p=mni2region([18,-62,22],atlas)
print(p)
coord2coord([18,-62,22],np.linalg.inv(atlas.affine))
#region_information(p-1)

86


array([36, 32, 47])

In [5]:
print(mni2region([18,0,56],atlas))
print(mni2cl_region([18,0,56],atlas))
print(mni2cl_region([-34,-4,5],atlas))

0
1
[29]
[168]


In [6]:
Networks1={}

In [16]:
for i in range(268):
    net=region_information(i)['Network']
    Networks1[net]=[]
for i in range(268):
    net=region_information(i)['Network']
    Networks1[net]=Networks1[net]+[i+1]

In [17]:
Networks1['Default']

[1,
 5,
 6,
 10,
 12,
 13,
 42,
 48,
 49,
 51,
 52,
 53,
 63,
 64,
 83,
 85,
 86,
 90,
 93,
 95,
 100,
 120,
 134,
 138,
 139,
 140,
 141,
 145,
 146,
 148,
 149,
 151,
 182,
 183,
 185,
 186,
 187,
 188,
 190,
 191,
 197,
 198,
 203,
 219,
 222,
 225,
 226,
 227,
 229,
 230,
 233]

In [ ]:
Networks1 #Dictionary of networks 

In [ ]:
Networks1.keys()

### Labelling SHEN regions using aal Atlas

In [10]:
#atlas.affine goes from voxel to mni

In [11]:
path_atlas="../Data/Parcellation/shen_2mm_268_parcellation.nii.gz"
shen_atlas=nib.load(path_atlas)

path_atlas="../../../Documentos/Software/mricron/Resources/templates/aal.nii.gz"
aal_atlas=nib.load(path_atlas)

NameError: name 'nib' is not defined

In [ ]:
aal_aff=aal_atlas.affine
shen_aff=shen_atlas.affine
#Aff=np.matmul(aal_aff,np.linalg.inv(shen_aff))

In [ ]:
a1=[26,28,33]
a2=coord2coord(a1,shen_aff)
print(a2)
a3=coord2coord(a2,np.linalg.inv(aal_aff))
print(a3)

In [ ]:
print(aal_atlas.get_fdata()[a3[0],a3[1],a3[2]])
print(shen_atlas.get_fdata()[91-a1[0],a1[1],a1[2]])

In [ ]:
a_matx=aal_atlas.get_fdata()
s_matx=shen_atlas.get_fdata()

In [ ]:
regions={}
for i in np.arange(1,269):
    regions[i]=list()

In [ ]:
dims=np.shape(s_matx)
I=dims[0]
J=dims[1]
K=dims[2]
for i in range(I):
    print(i)
    for j in range(J):
        for k in range(K):
            reg_1=s_matx[i,j,k] #region in Shen of voxel [i,j,k]
            if reg_1 >0 :
                a=coord2coord([i,j,k],shen_aff) #to mni
                b=coord2coord(a,np.linalg.inv(aal_aff)) #to voxel in aal
                reg_2=a_matx[b[0],b[1],b[2]] #region in aal 
                regions[reg_1]=regions[reg_1]+[(reg_2)] #save region

In [ ]:
with open("../Data/Parcellation/aal_atlas.json", "r") as read_file:
    names = json.load(read_file)

In [ ]:
i=85
for x in set(regions[i]):
    print(x,round((regions[i].count(x)/len(regions[i]))*100))

In [ ]:
regions_with_names={}
for key in regions.keys():
    regions_with_names[str(key)]={}
    l=len(regions[key])
    for x in set(regions[key]):
        if x>0:
            perc=round(regions[key].count(x)/l*100)
            if perc > 10:
                regions_with_names[str(key)][names[str(int(x))]]=str(perc)+ ' %'

In [9]:
#with open("../Data/Parcellation/labeling_with_aal.json", "w") as write_file:
#    json.dump(regions_with_names, write_file)

NameError: name 'regions_with_names' is not defined

In [ ]:
regions_with_names['29']

### Labelling SHEN regions using aal Atlas 2 (full names)

The file aal_regions.csv contains the short and full names of the regions. <br>
We create reg_with_full from regions_with_names 

In [177]:
with open("../Data/Parcellation/labeling_with_aal.json", "r") as read_file:
    content = json.load(read_file)

In [178]:
content['67']

{'Cerebelum_6_R': '31%', 'Occipital_Inf_R': '20%', 'Fusiform_R': '33%'}

In [179]:
full_names=pd.read_csv("../Data/Parcellation/aal_regions.csv",sep=',')
abbr=full_names['Unnamed: 1']
full=full_names['Unnamed: 2']
print(abbr[5])
print(full[5])

Frontal_Sup_Orb_L
Superior frontal gyrus, orbital
Part L


In [180]:
reg_with_full={} #new dict
for key in content.keys(): 
    print(key)
    reg_with_full[key]={} 
    for abb in content[key].keys(): #for region 
        index= abbr[abbr == abb]
        if (len(index)>0):
            ind= index.index[0] #index of region
            fulll=full[ind]
            fulll=str(fulll).replace("\n", " ")
            reg_with_full[key][fulll]=content[key][abb] #Replace with full name
        else:
            reg_with_full[key][abb]=content[key][abb] #Maintain name if not found

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268


In [181]:
reg_with_full['7']

{'Superior frontal gyrus, orbital Part R': '11%',
 'Middle frontal gyrus R': '25%',
 'Middle frontal gyrus, orbital Part R': '46%'}

In [182]:
with open("../Data/Parcellation/labeling_with_full_aal.json", "w") as write_file:
    json.dump(reg_with_full, write_file)